In [1]:
import pandas as pd
import os
import xml.etree.ElementTree as ET

In [2]:
data = pd.read_csv('/home/ubuntu/data/annotations/list.txt', header=None, sep=' ', skiprows=6)
data.columns = ['Image', 'CLASS_ID', 'SPECIES', 'BREED_ID']

In [3]:
xml_dir = '/home/ubuntu/data/annotations/xmls/'
images_with_boxes = os.listdir(xml_dir)
images_with_boxes = [n[:-4] for n in images_with_boxes] 

In [4]:
data = data.loc[data.Image.isin(images_with_boxes)]

In [5]:
validation = data.groupby('CLASS_ID', group_keys=False).apply(lambda x: x.sample(n=40, replace=False))
validation.sort_index(inplace=True)

In [6]:
train = data.drop(validation.index, axis=0)

In [7]:
len(train)

2191

In [8]:
len(validation)

1480

In [9]:
def write(df, filename):
    file = open(filename, 'w')
    for T in df.itertuples():
        img_name = T.Image + '.jpg'
        xml_name = T.Image + '.xml'
        file.write(img_name + ' ')
        tree = ET.parse(os.path.join(xml_dir, xml_name))
        annotations = []
        for child in tree.getroot():
            if child.tag == 'object':
                bbox = child.find('bndbox')
                xmin = bbox.find('xmin').text
                ymin = bbox.find('ymin').text
                xmax = bbox.find('xmax').text
                ymax = bbox.find('ymax').text
                class_label = T.CLASS_ID
                annotations.append(
                    '%s %s %s %s %s' % (xmin, ymin, xmax, ymax, class_label)
                )
        file.write('%d %s\n' % (len(annotations), ' '.join(annotations)))
    file.close()

In [10]:
write(train, 'PET_train_by_image.txt')
write(validation, 'PET_validation_by_image.txt')